# 7 - Struphy data structures

In this tutorial we will learn about the data structures used in Struphy to store FEEC and particle variables. 

For FEEC variables, Struphy relies on the distributed data structures provided by [Psydac](https://github.com/pyccel/psydac). In particular we need to understand the objects [StencilVector](https://github.com/pyccel/psydac/blob/652b29fdb813dce9836b34c0fd789c99cf779f17/psydac/linalg/stencil.py#L379) and [StencilMatrix](https://github.com/pyccel/psydac/blob/652b29fdb813dce9836b34c0fd789c99cf779f17/psydac/linalg/stencil.py#L859).

Regarding particle (PIC) variables, we will look at Struphy's [markers attribute](https://struphy.pages.mpcdf.de/struphy/sections/developers.html#struphy.pic.base.Particles.markers) of the [Particle class](https://struphy.pages.mpcdf.de/struphy/sections/developers.html#particle-base-class), which is just a static 2D numpy array. The communication of particles from one process to anther is done via the method [mpi_sort_markers](https://struphy.pages.mpcdf.de/struphy/sections/developers.html#struphy.pic.base.Particles.mpi_sort_markers).

## FEEC data structures

The [3D Derham sequence](https://struphy.pages.mpcdf.de/struphy/sections/discretization.html#id3) has two scalar-valued spaces, namely $H^1$ and $L^2$, and two vector-valued spaces, namely $H$(curl) and $H$(div). In the discrete case, members of $V^0_h\subset H^1$ and of $V^3_h \subset L^2$ are characterized by their FE coefficients in $\mathbb R^{N_0}$ and $\mathbb R^{N_3}$, respectively, and members $V^1_h\subset H$(curl) and of $V^2_h \subset H$(div) are characterized by their FE coefficients in $\mathbb R^{N_{1, 1} + N_{1,2} + N_{1,3}}$ and $\mathbb R^{N_{2, 1} + N_{2,2} + N_{2,3}}$. The scalar-valued coefficients are stored in [StencilVector](https://github.com/pyccel/psydac/blob/652b29fdb813dce9836b34c0fd789c99cf779f17/psydac/linalg/stencil.py#L379) format, whereas the vector-valued coefficients are stored in [BlockVector](https://github.com/pyccel/psydac/blob/652b29fdb813dce9836b34c0fd789c99cf779f17/psydac/linalg/block.py#L153) format. A `BlockVector` is nothing else than a 3-list of `StencilVectors`, so it is enough to understand the `StencilVector` object.

`StencilVectors` are elements of a [StencilVectorSpace](https://github.com/pyccel/psydac/blob/652b29fdb813dce9836b34c0fd789c99cf779f17/psydac/linalg/stencil.py#L84). A linear mapping between two `StencilVectorSpaces` can be represented by a [StencilMatrix](https://github.com/pyccel/psydac/blob/652b29fdb813dce9836b34c0fd789c99cf779f17/psydac/linalg/stencil.py#L859), which is a special kind of [LinearOperator](https://github.com/pyccel/psydac/blob/652b29fdb813dce9836b34c0fd789c99cf779f17/psydac/linalg/basic.py#L193). For the vector-valued equivalent [BlockVectorSpace](https://github.com/pyccel/psydac/blob/652b29fdb813dce9836b34c0fd789c99cf779f17/psydac/linalg/block.py#L18) the corresponding matrix is a [BlockLinearOperator](https://github.com/pyccel/psydac/blob/652b29fdb813dce9836b34c0fd789c99cf779f17/psydac/linalg/block.py#L462), which is nothing else than a nested list of `StencilMatrices`. Thus, it is enough to understand the `StencilMatrix` object.

### StencilVector - serial case

`StencilVectors` are initialized from a [StencilVectorSpace](https://github.com/pyccel/psydac/blob/652b29fdb813dce9836b34c0fd789c99cf779f17/psydac/linalg/stencil.py#L84). In Struphy, all necessary `StencilVectorSpaces` can be read out from the [Derham](https://struphy.pages.mpcdf.de/struphy/sections/developers.html?highlight=derham#derham-sequence-3d) object (for an in-dpeth view on the `Derham` class please go to the API [discrete de Rham sequence](https://struphy.pages.mpcdf.de/struphy/api/discrete_derham.html#API:-Discrete-de-Rham-sequence)).

In [ ]:
from psydac.linalg.stencil import StencilVector
from struphy.feec.psydac_derham import Derham
import numpy as np

Nel = [8, 8, 12]  # number of elements
p = [2, 3, 4]  # spline degrees
# spline boundary conditions (periodic=True, clamped=False)
spl_kind = [False, False, True]

# Psydac discrete Derham sequence
dr_serial = Derham(Nel, p, spl_kind)

# element of V0_h
x0 = StencilVector(dr_serial.Vh['0'])

assert np.all(x0[:] == 0.)

We can see that a `StencilVector` is initialized with zeros. Moreover, it can be [indexed and sliced](https://numpy.org/doc/stable/user/basics.indexing.html) like a numpy array:

In [ ]:
print(f'{type(x0) = }')
print(f'{type(x0[:])              = }')
print(f'{type(x0[:, :, :])        = }')
print(f'{type(x0[:2, 1:2:7, :-1]) = }')

With this, we can set specific entries in the `StencilVector`:

In [ ]:
print(f'{x0[3, 2, 1] = }')
x0[3, 2, 1] = 99.
print(f'{x0[3, 2, 1] = }')

When indexing a `StencilVector`, it important to note that **global indices (!)** have to be used. These are the indices of the full (global) 3D array, one could call them "mathematical indices". This will become important when we run in parallel below. The global indices available on the current process can be obtained from the attributes of the `StencilVector`:

In [ ]:
from struphy.utils.utils import print_all_attr

print_all_attr(x0)

The global start and end indices in each direction are:

In [ ]:
print(f'{x0.starts = }')
print(f'{x0.ends = }')

Let us explain why these values make sense. The dimension of a 1D-periodic spline space is `Nel`, and of a 1D-clamped spline space it is `Nel + p`. In our case:

In [ ]:
dims = [Ni + pi*(not spi) for Ni, pi, spi in zip(Nel, p, spl_kind)]
print(f'{dims                    = }')
print(f'{dims[0]*dims[1]*dims[2] = }' + ' = total dimension of vector space')

Because we are running on just one process at the moment (no domain decomposition), the start index is zero and the end index is `dims - 1` in each direction, exactly what is is given by `x0.starts` and `x0.ends`.

The data of a `StencilVector` is stored in a `numpy` array. This array can be accessed in different ways:

In [ ]:
print(f'{type(x0[:])       = }, {np.shape(x0[:])       = }')
print(f'{type(x0[:, :, :]) = }, {np.shape(x0[:, :, :]) = }')
print(f'{type(x0._data)    = }, {np.shape(x0._data)    = }')

Actually, these are not the same arrays in memory, but copies of each other:

In [ ]:
print(f'{id(x0[:])       = }')
print(f'{id(x0[:, :, :]) = }')
print(f'{id(x0._data)    = }')

Let us try to understand the `shape` of these numpy arrays (printed above). Indeed, these shapes do not correspond to `dims`, as one could have expected. This is because the numpy arrays also contain the `ghost regions` needed for MPI communication. In each of the three directions, there is a ghost region of size `p` attached to the left and to the right of the actual domain. Thus, we expect the array size to be `dim + 2*p` in each direction. Let's check: 

In [ ]:
shape = [dim + 2*pi for dim, pi in zip(dims, p)]
print(f'{shape = }')

The presence of ghost regions needs to be taken into account when `indexing` a `StencilVector`. For example, if we want to write a vector `a` into our `StencilVector`, we have to be careful with the `slicing`:  

In [ ]:
a = np.arange(dims[0] * dims[1] * dims[2]).reshape(*dims)
x0[:] = 99.

s = x0.starts
e = x0.ends
x0[s[0]: e[0] + 1, s[1]: e[1] + 1, s[2]: e[2] + 1] = a
print(f'{x0[0, 0, :4] = }')
print(f'{x0[0, :4, 0] = }')
print(f'{x0[:4, 0, 0] = }')

Note that the output after slicing `:4` is an array of size 8, which contains the ghost regions. This is because the ghost regions are addressed with negative indices. Let us verify:

In [ ]:
x0[0, 0, -1] = 11
print(f'{x0[0, 0, :4] = }')
print(f'{x0[0, 0, 0:4] = }')

There is also a way to use **local indices** for addressing a `StencilVector`, namely by writing directly into the `_data` attribute. In this case the usual numpy indexing of the array applies, **and the ghost regions have to be taken into account explicitly (!)** via the `pads`. The `pads` attribute holds the size of the ghost regions, which is always `p`:

In [ ]:
x0[0, 0, -1] = 99.
y0 = StencilVector(dr_serial.Vh['0'])
y0[:] = 99.

pd = y0.pads
print(f'{pd = }')
y0._data[pd[0]: -pd[0], pd[1]: -pd[1], pd[2]: -pd[2]] = a
print(f'{y0[0, 0, :4] = }')
print(f'{y0[0, :4, 0] = }')
print(f'{y0[:4, 0, 0] = }')

assert np.all(x0[:] == y0[:])

Finally, let us inspect the `to_array()` method of a `StencilVector`:

In [ ]:
print(f'{x0.shape                 = }')
print(f'{dims[0]*dims[1]*dims[2]  = }')
print(f'{type(x0.toarray())       = }')
print(f'{x0.toarray().shape       = }')

The result of `toarray()` is obviously a flattened version of the 3D numpy array holding the data, **without ghost regions (!)**. The ordering is `row-major` (C-style):

In [ ]:
flat_data = x0[s[0]: e[0] + 1, s[1]: e[1] + 1, s[2]: e[2] + 1].flatten()
assert np.all(x0.toarray() == flat_data)
print(f'{x0.toarray()[:4]  = }')
print(f'{x0.toarray()[-4:] = }')

### StencilMatrix - serial case

The initialization is similar to a `StencilVector`, but this time we have to state the `domain` and `codomain` of the linear mapping:

In [ ]:
from psydac.linalg.stencil import StencilMatrix

A0 = StencilMatrix(dr_serial.Vh['0'], dr_serial.Vh['0'])

assert np.all(A0[:, :] == 0.)

We can see that a `StencilMatrix` is initialized with zeros. Moreover, it can be [indexed and sliced](https://numpy.org/doc/stable/user/basics.indexing.html) like a numpy array:

In [ ]:
print(f'{type(A0) = }')
print(f'{type(A0[:, :])                      = }')
print(f'{type(A0[:, :, :, :, :, :])          = }')
print(f'{type(A0[:2, 1:2:7, :-1, :, 2, :])   = }')

With this, we can set specific entries in the `StencilMatrix`:

In [ ]:
print(f'{A0[3, 2, 1, 0, 0, 0] = }')
A0[3, 2, 1, 0, 0, 0] = 99.
print(f'{A0[3, 2, 1, 0, 0, 0] = }')

When indexing a `StencilMatrix`, it important to note that **global indices (!)** have to be used **for the row indices (!)**. These are the indices of the rows of the full (global) 3D array, one could call them "mathematical row indices". This will become important when we run in parallel below. The global row-indices available on the current process can be obtained from the attribute `codomain` of the `StencilMatrix`:

In [ ]:
print_all_attr(A0)

In [ ]:
print(f'{A0.codomain.starts = }')
print(f'{A0.codomain.ends = }')

The `row-indexing` works pretty much as for the `StencilVector` described above. The characteristic feature of a `StencilMatrix` is the column storage. Namely, it stores only `2*p + 1` (off-)diagonals for each direction:

In [ ]:
print(f'{[2*pi + 1 for pi in p]     = }')
print(f'{A0[0, 0, 0, :, :, :].shape = }')

The column index runs between `[-p, p]` and the diagonal is at `0` (!), which means that lower diagonals are addressed with negativ indices:

In [ ]:
s = A0.codomain.starts
e = A0.codomain.ends

for n in range(- p[2], p[2] + 1):
    A0[0, 0, s[2]: e[2] + 1, :, :, n] = n*10

print('A0[0, 0, :, 0, 0, :] = ')
print(A0[0, 0, :, 0, 0, :])

Note the ghost regions in the row index, as in the case of the `StencilVector`. 

Beware that **the above output is not the actual matrix (!)**, but merely the array in which the (off-)diagonals are stored. The actual matrix can be obtained from `toarray()`. For simplicity, we shall show this in the 1D case:

In [ ]:
vector_space_1d = dr_serial.Vh_fem['0'].spaces[2].coeff_space
A0_1d = StencilMatrix(vector_space_1d, vector_space_1d)

s = A0_1d.codomain.starts
e = A0_1d.codomain.ends

for n in range(- p[2], p[2] + 1):
    A0_1d[s[0]: e[0] + 1, n] = n*10

print('A0_1d[0, 0, :, 0, 0, :] = ')
print(A0_1d[:, :])

print('\nA0_1d.toarray() = ')
print(A0_1d.toarray())

Much like for the `StencilVector`, there is also a way to use **local indices** for addressing a `StencilMatrix`, namely by writing directly into the `_data` attribute. In this case the usual numpy indexing of the array applies, **and the ghost regions have to be taken into account explicitly (!)** via the `pads`. In contrast to the global indexing, the columns are addressed with indices between `[0, 2*p]`, where the diagonal is at `p` (and not at `0`):

In [ ]:
a = np.arange(dims[0] * dims[1] * dims[2]).reshape(*dims)

A0[:, :] = 99.

s = A0.codomain.starts
e = A0.codomain.ends
pd = A0.pads

A0[s[0]: e[0] + 1, s[1]: e[1] + 1, s[2]: e[2] + 1, 0, 0, 0] = a

B0 = StencilMatrix(dr_serial.Vh['0'], dr_serial.Vh['0'])

B0[:, :] = 99.

s = B0.codomain.starts
e = B0.codomain.ends
pd = B0.pads

B0._data[pd[0]: -pd[0], pd[1]: -pd[1], pd[2]: -pd[2], p[0], p[1], p[2]] = a

assert np.all(A0[:, :] == B0[:, :])

Finally, let us inspect the `to_array()` method of a `StencilMatrix`:

In [ ]:
print(f'{A0.shape                 = }')
print(f'{dims[0]*dims[1]*dims[2]  = }')
print(f'{type(A0.toarray())       = }')
print(f'{A0.toarray().shape       = }')

The result of `toarray()` is the full matrix, with the domain and codomain flattened in `row-major` (C-style), and **without ghost regions (!)**.

### StencilVector - parallel case

The distributed `StencilVector` can be understood via the following sketch:

![StencilVector](../pics/stencil_vec.jpg)

If we want to show the distributed character of the `StencilVector` in an example, we must run with `ipyparallel`:

In [ ]:
import ipyparallel as ipp


def stencil_vec_shape():

    from struphy.feec.psydac_derham import Derham
    from psydac.linalg.stencil import StencilVector
    from psydac.ddm.mpi import mpi as MPI
    import numpy as np

    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()

    Nel = [8, 8, 12]  # number of elements
    p = [2, 3, 4]  # spline degrees
    # spline boundary conditions (periodic=True, clamped=False)
    spl_kind = [False, False, True]

    dr = Derham(Nel, p, spl_kind, comm=comm)

    x0 = StencilVector(dr.Vh['0'])

    assert np.all(x0[:] == 0.)

    out = f'{rank = }, {x0.starts = }, {x0.ends = }, {x0.pads = }, {np.shape(x0[:]) = }:'

    return out


with ipp.Cluster(engines='mpi', n=2) as rc:
    view = rc.broadcast_view()
    r = view.apply_sync(stencil_vec_shape)
    print("\n".join(r))

Here, we can clearly see the parallelization (domain decomposition) in the third direction, manifest in the different `starts` and `ends` on each process. These are the global, or "mathematical" indices. The `shape` of 14 in the third direction can be understood as follows:

The total dimension in the third direction is 12. This has been split into two equal parts by the domain decomposition, meaning that coefficients with indices `[0, 5]` are stored on the first process, and coefficients with indices `[6, 11]` are stored on the second process. The size of the ghost regions (`pads`) is four in both cases, yielding 4 + 6 + 4 = 14.  

Next, let us look at the MPI communication via `update_ghost_regions()`:

In [ ]:
def stencil_vec_ghost():

    from struphy.feec.psydac_derham import Derham
    from psydac.linalg.stencil import StencilVector
    from psydac.ddm.mpi import mpi as MPI
    import numpy as np

    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()

    Nel = [4, 4, 12]  # number of elements
    p = [2, 2, 2]  # spline degrees
    # spline boundary conditions (periodic=True, clamped=False)
    spl_kind = [False, False, True]

    dr = Derham(Nel, p, spl_kind, comm=comm)

    x0 = StencilVector(dr.Vh['0'])
    s = x0.starts
    e = x0.ends
    pd = x0.pads

    assert np.all(x0[:] == 0.)

    x0[:] = -99.
    x0[s[0], s[1], s[2]: e[2] + 1] = np.arange(e[2] + 1 - s[2])*10**rank

    out = f'{rank = }, before update: {x0[s[0], s[1], :] = }:'

    x0.update_ghost_regions()

    out += f'\n{rank = }, after update:  {x0[s[0], s[1], :] = }:'

    return out


with ipp.Cluster(engines='mpi', n=3) as rc:
    view = rc.broadcast_view()
    r = view.apply_sync(stencil_vec_ghost)
    print("\n".join(r))

As we can see from the different ranks, the logic is exactly the one displayed in the picture above. The ghost regions accept the incoming data from the neighboring processes. Ghost regions do not send out data.

Finally, let us check the `toarray()` function in the parallel case:

In [ ]:
def stencil_vec_toarray():

    from struphy.feec.psydac_derham import Derham
    from psydac.linalg.stencil import StencilVector
    from psydac.ddm.mpi import mpi as MPI
    import numpy as np

    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()

    Nel = [8, 8, 12]  # number of elements
    p = [2, 3, 4]  # spline degrees
    # spline boundary conditions (periodic=True, clamped=False)
    spl_kind = [False, False, True]

    dr = Derham(Nel, p, spl_kind, comm=comm)

    x0 = StencilVector(dr.Vh['0'])

    assert np.all(x0[:] == 0.)

    out = f'{rank = }, {np.shape(x0.toarray()) = },  {np.shape(x0.toarray_local()) = }'

    return out


with ipp.Cluster(engines='mpi', n=2) as rc:
    view = rc.broadcast_view()
    r = view.apply_sync(stencil_vec_toarray)
    print("\n".join(r))

The `toarray_local()` gives only the (flattened) local data, whereas `toarray()` yields global size. In the latter case, data points that are not available on the current process are filled with zeros.

### StencilMatrix - parallel case

The distributed `StencilMatrix` can be understood via the following sketch:

![StencilMatrix](../pics/stencil_matrix.jpg)

It is important to note that only the row indices are distributed, and that they are distributed much like a `StencilVector`. The `2+p + 1` (off-)diagonals are stored in a numpy array. The indexing has been explained above.

If we want to show the distributed character of the `StencilMatrix` in an example, we must run with `ipyparallel`:

In [ ]:
def stencil_mat_shape():

    from struphy.feec.psydac_derham import Derham
    from psydac.linalg.stencil import StencilMatrix
    from psydac.ddm.mpi import mpi as MPI
    import numpy as np

    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()

    Nel = [8, 8, 12]  # number of elements
    p = [2, 3, 4]  # spline degrees
    # spline boundary conditions (periodic=True, clamped=False)
    spl_kind = [False, False, True]

    dr = Derham(Nel, p, spl_kind, comm=comm)

    A0 = StencilMatrix(dr.Vh['0'], dr.Vh['0'])

    assert np.all(A0[:, :] == 0.)

    out = f'{rank = }, {A0.codomain.starts = }, {A0.codomain.ends = }, {A0.pads = }, {np.shape(A0[:, :]) = }:'

    return out


with ipp.Cluster(engines='mpi', n=2) as rc:
    view = rc.broadcast_view()
    r = view.apply_sync(stencil_mat_shape)
    print("\n".join(r))

Here, we can clearly see the parallelization (domain decomposition) of the row indices in the third direction, manifest in the different `starts` and `ends` on each process. These are the global, or "mathematical" indices. The `shape` of 14 in the third direction has been explained above. The column dimensions are `2*p + 1`. 

Next, let us look at the MPI communication via `update_ghost_regions()`:

In [ ]:
def stencil_mat_ghost():

    from struphy.feec.psydac_derham import Derham
    from psydac.linalg.stencil import StencilMatrix
    from psydac.ddm.mpi import mpi as MPI
    import numpy as np

    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()

    Nel = [4, 4, 12]  # number of elements
    p = [2, 2, 2]  # spline degrees
    # spline boundary conditions (periodic=True, clamped=False)
    spl_kind = [False, False, True]

    dr = Derham(Nel, p, spl_kind, comm=comm)

    A0 = StencilMatrix(dr.Vh['0'], dr.Vh['0'])
    s = A0.codomain.starts
    e = A0.codomain.ends
    pd = A0.pads

    assert np.all(A0[:, :] == 0.)

    A0[:, :] = -99.
    A0[s[0], s[1], s[2]: e[2] + 1, 0, 0, -pd[2]: pd[2] + 1] = np.arange(
        (e[2] + 1 - s[2])*(2*pd[2] + 1)).reshape(e[2] + 1 - s[2], 2*pd[2] + 1)*10**rank

    out = f'{rank = }, before update: A0[s[0], s[1], :, 0, 0, :] = \n{A0[s[0], s[1], :, 0, 0, :]}:'

    A0.update_ghost_regions()

    out += f'\n{rank = }, after update: A0[s[0], s[1], :, 0, 0, :] = \n{A0[s[0], s[1], :, 0, 0, :]}:'

    return out


with ipp.Cluster(engines='mpi', n=2) as rc:
    view = rc.broadcast_view()
    r = view.apply_sync(stencil_mat_ghost)
    print("\n".join(r))

As we can see from the different ranks, the logic for the row indices is exactly the one displayed for the `StencilVector`. The ghost regions accept the incoming data from the neighboring processes. Ghost regions do not send out data.

Finally, let us check the `toarray()` function in the parallel case:

In [ ]:
def stencil_mat_toarray():

    from struphy.feec.psydac_derham import Derham
    from psydac.linalg.stencil import StencilMatrix
    from psydac.ddm.mpi import mpi as MPI
    import numpy as np

    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()

    Nel = [8, 8, 12]  # number of elements
    p = [2, 3, 4]  # spline degrees
    # spline boundary conditions (periodic=True, clamped=False)
    spl_kind = [False, False, True]

    dr = Derham(Nel, p, spl_kind, comm=comm)

    A0 = StencilMatrix(dr.Vh['0'], dr.Vh['0'])

    assert np.all(A0[:, :] == 0.)

    out = f'{rank = }, {np.shape(A0.toarray()) = }'

    return out


with ipp.Cluster(engines='mpi', n=2) as rc:
    view = rc.broadcast_view()
    r = view.apply_sync(stencil_mat_toarray)
    print("\n".join(r))

The `toarray()` method gives the (flattened) global size. Data points that are not available on the current process are filled with zeros.

## PIC data structures

In Struphy all PIC related information is stored in the base class [Particles](https://struphy.pages.mpcdf.de/struphy/sections/developers.html#struphy.pic.base.Particles). In particular, particles are stored in [Particles.markers](https://struphy.pages.mpcdf.de/struphy/sections/developers.html#struphy.pic.base.Particles.markers), which is a simple 2D numpy array. The row index refers to the particles and the column index to the attributes of particles (coordinates, weights, etc.). 

The `markers`-array is a static array with more rows than the actual number of particles on a process, because of redistribution of particles when positions change (domain decomposition). Markers sent to another process leave behind a "hole", which can then be filled by another incoming marker. The marker communication is accomplished by [Particles.mpi_sort_markers](https://struphy.pages.mpcdf.de/struphy/sections/developers.html#struphy.pic.base.Particles.mpi_sort_markers).

### The markers array

We start by looking at the different particle classes that are available in Struphy:

In [ ]:
import sys, inspect
from struphy.pic import particles

for name, obj in inspect.getmembers(particles):
    if inspect.isclass(obj) and obj.__module__ == particles.__name__:
        print(obj)

We now create an instance of `Particles6D` with default parameters:

In [ ]:
from struphy.pic.particles import Particles6D
from struphy.pic.utilities import LoadingParameters

loading_params = LoadingParameters(Np=120)
particles = Particles6D(loading_params=loading_params)

particles.draw_markers()

Some important attributes are: the total number of markers, the shape of the markers array, and the shape of the array with "holes" removed:

In [ ]:
print(f'{particles.Np                     = }')
print(f'{particles.markers.shape          = }')
print(f'{particles.markers_wo_holes.shape = }')

Let us look at some parameters/coordinates of the first five markers:

In [ ]:
print(f'{particles.positions[:5] = }\n')
print(f'{particles.velocities[:5] = }\n')
print(f'{particles.phasespace_coords[:5] = }\n')
print(f'{particles.weights[:5] = }\n')
print(f'{particles.sampling_density[:5] = }\n')
print(f'{particles.weights0[:5] = }\n')
print(f'{particles.marker_ids[:5] = }')